In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
sys.path.append("../extraction_core/page_selection")
sys.path.append("../extraction_core/extraction")
sys.path.append("../extraction_core/")

In [2]:
import hvac
import os
client = hvac.Client(
    url="https://vault.lab.sspcloud.fr", token=os.environ["VAULT_TOKEN"]
)

secret = os.environ["VAULT_MOUNT"] + os.environ["VAULT_TOP_DIR"] + "/s3_creds"
mount_point, secret_path = secret.split("/", 1)
secret_dict = client.secrets.kv.read_secret_version(
    path=secret_path, mount_point=mount_point
)

os.environ["AWS_ACCESS_KEY_ID"] = secret_dict["data"]["data"][
    "ACCESS_KEY"
]
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_dict["data"]["data"][
    "SECRET_KEY"
]
try:
    del os.environ["AWS_SESSION_TOKEN"]
except KeyError:
    pass

In [4]:
from extraction.data import fs

fs.ls("projet-extraction-tableaux/orange/bmp")

['projet-extraction-tableaux/orange/bmp/306769688.bmp',
 'projet-extraction-tableaux/orange/bmp/307299248.bmp',
 'projet-extraction-tableaux/orange/bmp/345039416.bmp',
 'projet-extraction-tableaux/orange/bmp/379984891.bmp',
 'projet-extraction-tableaux/orange/bmp/380129866.bmp',
 'projet-extraction-tableaux/orange/bmp/384518114.bmp',
 'projet-extraction-tableaux/orange/bmp/409759156.bmp',
 'projet-extraction-tableaux/orange/bmp/430107359.bmp',
 'projet-extraction-tableaux/orange/bmp/432668432.bmp',
 'projet-extraction-tableaux/orange/bmp/440419240.bmp',
 'projet-extraction-tableaux/orange/bmp/500413505.bmp',
 'projet-extraction-tableaux/orange/bmp/500440813.bmp',
 'projet-extraction-tableaux/orange/bmp/501614572.bmp']

In [6]:
from pathlib import Path

for path in fs.ls("projet-extraction-tableaux/orange/bmp"):
    file_name = Path(path).stem
    fs.get(path, "../data/orange/" + file_name + ".bmp")

In [19]:
from PIL import Image
from extraction.table_extractor import TableExtractor
import numpy as np

In [9]:
table_extractor = TableExtractor.from_checkpoint(
    "projet-extraction-tableaux/logs/TableNetModule/version_00/checkpoints/marmot_model.ckpt",
    config="--oem 3 --psm 11"
)

siren_list = [
    "306769688",
    "307299248",
    "345039416",
    "379984891",
    "380129866",
    "384518114",
    "409759156",
    "430107359",
    "432668432",
    "440419240",
    "500413505",
    "500440813",
    "501614572"
]
data_dir = "../data/orange"
test_images = [
    Path(data_dir).joinpath(path + ".bmp") for path in siren_list
]

extracted_tables = {}
clean_tables = {}

for siren, image_path in zip(siren_list, test_images):
    extracted_tables[siren] = []
    image = Image.open(image_path)
    out = table_extractor.extract(image)

    for i, df in enumerate(out["tables"]):
        extracted_tables[siren].append(df)

Lightning automatically upgraded your loaded checkpoint from v1.1.6 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file weights.ckpt`
/home/onyxia/work/extraction-comptes-sociaux/notebooks/../extraction_core/extraction/column_assembler.py:39: RuntimeWarning: Mean of empty slice.
  (df.top.unique()[1:] - df.top.unique()[:-1]).mean()
/opt/mamba/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/onyxia/work/extraction-comptes-sociaux/notebooks/../extraction_core/extraction/column_assembler.py:39: RuntimeWarning: Mean of empty slice.
  (df.top.unique()[1:] - df.top.unique()[:-1]).mean()
/opt/mamba/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/onyxia/work/extraction-comptes-sociaux/notebooks/../extr

In [10]:
idx = 0
siren = siren_list[idx]

len(extracted_tables[siren])

1

In [11]:
for table in extracted_tables[siren]:
    print(table)

                                                  col                 col  \
line_id                                                                     
1                          Filiales et participations                 NaN   
2                                        Exprimé en €                 NaN   
3                          Filiales et participations  Valeurs comptables   
4                                                 NaN                 NaN   
5                                                 NaN                 NaN   
6                                                 NaN               Brute   
7                             A. Renseignements dé es                 NaN   
8                           Orange Capital Management           3 342 864   
9                     Orange Digital Ventures Support          68 200 000   
10                     Orange Digital Ventures Africa                 152   
11                                               OVCI           2 377 768   

In [14]:
fields = [
    "Quote-part du capital détenue (en %)",
    "Valeur brute comptable des titres détenus (en K euros)",
    "Valeur nette comptable des titres détenus (en K euros)",
    "Dividendes encaissés par la société en cours d'exercice (en K euros)",
    "Résultats (bénéfices ou pertes à 100%) "
    "du dernier exercice clos (en K devises)",
]

# Liste des pattern regex afin de déterminer les colonnes d'intérêt
regexes = [
    r"\b(quote-part){e<2}\b",
    r"\b(brute){e<2}\b",
    r"\b(nette){e<2}\b",
    r"\b(dividende){e<4}\b",
    r"\b(resultat){e<3}\b",
]

In [64]:
from typing import List
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
import regex


class NewTableCleaner:
    """
    Cleaner for the table output by `TableExtractor.extract`.
    """

    def __init__(
        self,
        colnames_out: List[str],
        regexes: List[str],
        pct_digit: float = 0.2,
    ):
        """
        Constructor for the TableCleaner class.

        Args:
            colnames_out (List[str]): Columns desired in the output.
            regexes (List[str]): Regular expressions to find in column names.
            pct_digit (float): Minimum percentage of digits in a row (with
                the first column removed) to have it considered as numeric.
        """
        self.colnames_out = colnames_out
        self.regexes = regexes
        self.pct_digit = pct_digit

    def clean_df(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Selects relevant columns in the pd.DataFrame given as input and
        returns a filtered pd.DataFrame with a clean index and clean
        column names.

        Args:
            df (pd.DataFrame): DataFrame to clean.
        """
        df_without_first_column = df.iloc[:, 1:]
        row_types = self.get_row_types(df_without_first_column)

        if not row_types:
            return df

        # First row can never be values already
        row_types[0] = False
        if not any(row_types):
            header_rows = np.arange(len(row_types))
            body_rows = np.array([])
        else:
            header_rows = np.arange(np.where(row_types)[0][0])
            body_rows = np.arange(np.where(row_types)[0][0], df.shape[0])

        header_df = df.iloc[header_rows]
        body_df = df.iloc[body_rows]

        similarities = []
        fuzzy_matches = []

        for name, value in header_df.items():
            value_cleaned = value.dropna()
            content = " ".join(value_cleaned)
            similarities.append(
                np.array(
                    [
                        self.similar(content, field)
                        for field in self.colnames_out
                    ]
                )
            )
            fuzzy_matches.append(
                np.array(
                    [
                        1
                        if regex.search(
                            regex_pattern, content, flags=regex.IGNORECASE
                        )
                        else 0
                        for regex_pattern in self.regexes
                    ]
                )
            )

        similarities = np.stack(similarities)
        fuzzy_matches = np.stack(fuzzy_matches)

        N, M = similarities.shape
        # Arrays of shape (N, M) with N the number of columns
        # in the extracted table and M = 6 (number of columns we
        # want to match)

        if N > M:
            column_ids = self.ids_from_long_similarities(similarities, fuzzy_matches)
        else:
            column_ids = self.ids_from_wide_similarities(similarities, fuzzy_matches)

        body_df.columns = column_ids
        
        return body_df

    def get_row_types(self, df: pd.DataFrame) -> List[bool]:
        """
        From an input DataFrame, returns a list of booleans and of
        length the number of rows in the DataFrame. Each element of this
        list is set to True if the corresponding row has a percentage of
        digit characters greater than pct_digit and False otherwise.

        Args:
            df (pd.DataFrame): DataFrame to get row types for.
        """
        row_types = []

        for row_index, row in df.iterrows():
            row_cleaned = row.dropna()
            row_as_string = "".join(row_cleaned)

            numbers = sum(c.isdigit() for c in row_as_string)

            try:
                if numbers / len(row_as_string) > self.pct_digit:
                    row_types.append(True)
                else:
                    row_types.append(False)
            except ZeroDivisionError:
                row_types.append(False)

        return row_types

    def ids_from_wide_similarities(
        self,
        similarities: np.ndarray,
        fuzzy_matches: np.ndarray,
    ) -> List[str]:
        """
        Get column ids from wide similarity and fuzzy
        match matrices.

        Args:
            similarities (np.ndarray): Similarity matrix of shape N x M.
            fuzzy_matches (np.ndarray): Match matrix of shape N x M.
        Returns:
            List[str]: Column ids.
        """
        column_match = self.stable_match_wide(similarities * fuzzy_matches)
        scores = np.take_along_axis(
            similarities * fuzzy_matches,
            np.expand_dims(column_match, 1),
            axis=1,
        ).squeeze()

        # Array of length M, indicates position of columns IF1
        # to IF11 (in that order) in the extracted array
        column_match = np.where(scores, column_match, -1)

        column_ids = [""] * similarities.shape[0]
        for i, j in enumerate(column_match):
            if j >= 0:
                index = self.colnames_out[j]
                column_ids[i] = index

        return column_ids

    def ids_from_long_similarities(
        self,
        similarities,
        fuzzy_matches,
    ):
        """
        Get column ids from wide similarity and fuzzy
        match matrices.

        Args:
            similarities (np.ndarray): Similarity matrix of shape N x M.
            fuzzy_matches (np.ndarray): Match matrix of shape N x M.
        Returns:
            List[str]: Column ids.
        """
        column_match = self.stable_match_long(similarities * fuzzy_matches)
        scores = np.take_along_axis(
            similarities * fuzzy_matches,
            np.expand_dims(column_match, 0),
            axis=0,
        )

        # Array of length M, indicates position of columns IF1
        # to IF11 (in that order) in the extracted array
        column_match = np.where(scores, column_match, -1).squeeze()

        column_ids = [""] * similarities.shape[0]
        for i, j in enumerate(column_match):
            if j >= 0:
                index = self.colnames_out[i]
                column_ids[j] = index

        return column_ids
    
    @staticmethod
    def stable_match_long(similarities: np.array) -> np.array:
        """
        From a 2D matrix of similarities between the column names
        desired in the output and the column names in an input matrix,
        returns a stable matching.

        Args:
            similarities (np.array): Similarity matrix.
        """
        order = similarities.argsort(0)
        ncolumns_to_match = order.shape[1]
        ncolumns = order.shape[0]

        FREE = -1
        match = FREE * np.ones(ncolumns_to_match, dtype=np.int_)
        jnext = ncolumns * np.ones(ncolumns_to_match, dtype=np.int_)
        rev_match = FREE * np.ones(ncolumns, dtype=np.int_)

        while np.any(match == FREE):
            i = np.where(match == FREE)[0][0]
            jnext[i] -= 1
            j = order[jnext[i], i]
            if rev_match[j] == FREE:
                rev_match[j], match[i] = i, j
            else:
                if similarities[j, rev_match[j]] < similarities[j, i]:
                    match[rev_match[j]] = FREE
                    rev_match[j], match[i] = i, j

        return match

    @staticmethod
    def stable_match_wide(similarities: np.array) -> np.array:
        """
        From a 2D matrix of similarities between the column names
        desired in the output and the column names in an input matrix,
        returns a stable matching.

        Args:
            similarities (np.array): Similarity matrix.
        """
        transposed_similarities = similarities.T

        order = transposed_similarities.argsort(0)
        ncolumns_to_match = order.shape[1]
        ncolumns = order.shape[0]

        FREE = -1
        match = FREE * np.ones(ncolumns_to_match, dtype=np.int_)
        jnext = ncolumns * np.ones(ncolumns_to_match, dtype=np.int_)
        rev_match = FREE * np.ones(ncolumns, dtype=np.int_)

        while np.any(match == FREE):
            i = np.where(match == FREE)[0][0]
            jnext[i] -= 1
            j = order[jnext[i], i]
            if rev_match[j] == FREE:
                rev_match[j], match[i] = i, j
            else:
                if transposed_similarities[j, rev_match[j]] < transposed_similarities[j, i]:
                    match[rev_match[j]] = FREE
                    rev_match[j], match[i] = i, j

        return match

    @staticmethod
    def similar(a: str, b: str) -> float:
        """
        Returns the Levenshtein similarity between two strings given as inputs.

        Args:
            a (str): First string.
            b (str): Second string.
        """
        return SequenceMatcher(None, a, b).ratio()


In [91]:
import re

def replace_comma(col):
    return col.str.replace(",", r"\.")


def clean_string(s):
    if isinstance(s, str):
        pattern = r'[^0-9\.]'
        return re.sub(pattern, '', s)
    else:
        return s


def cast_column(col):
    return col.astype(str).str.strip().astype(float)

In [95]:
cleaner = NewTableCleaner(fields, regexes)
clean_tables = {}

for siren, table_list in extracted_tables.items():
    print("---")
    print(siren)
    selected_df = None
    num_cols = 0
    
    for df in table_list:
        # Clean
        clean_df = cleaner.clean_df(df)
        if clean_df is not None:
            first_column = clean_df.iloc[:, 0]
            df_to_filter = clean_df.iloc[:, 1:]

            filtered_df = df_to_filter.loc[:, ~df_to_filter.columns.isin([""])]
            if filtered_df.shape[1] > num_cols:
                num_cols = filtered_df.shape[1]
                # Clean up les colonnes numériques
                filtered_df = filtered_df.apply(replace_comma)
                filtered_df = filtered_df.applymap(clean_string)
                filtered_df = filtered_df.replace("", "0")
                filtered_df = filtered_df.replace(".", "0")
                filtered_df = filtered_df.replace('\.\.', '.', regex=True)
                print(filtered_df)
                filtered_df = filtered_df.apply(cast_column)
                filtered_df = filtered_df.fillna(0.0)
                selected_df = pd.concat([first_column, filtered_df], axis=1)
        
        clean_tables[siren] = selected_df

---
306769688
        Valeur brute comptable des titres détenus (en K euros)  \
line_id                                                          
8                                                  3342864       
9                                                 68200000       
10                                                     152       
11                                                 2377768       
12                                                26611500       
13                                                 1510578       
14                                                    8582       
15                                                10010000       
16                                                11510000       
17                                                55010000       
18                                                10510000       
19                                                   10000       
20                                                 3049110    

In [96]:
idx = 0
siren = siren_list[idx]

In [97]:
for table in extracted_tables[siren]:
    print(table.shape)
    print(table.head())

(49, 8)
                                col                 col  \
line_id                                                   
1        Filiales et participations                 NaN   
2                      Exprimé en €                 NaN   
3        Filiales et participations  Valeurs comptables   
4                               NaN                 NaN   
5                               NaN                 NaN   

                           col      col                            col  \
line_id                                                                  
1                          NaN      NaN                            NaN   
2                          NaN      NaN                            NaN   
3        comptables des titres  détenus  Quote- part du capital détenu   
4                          NaN      NaN                            NaN   
5                          NaN      NaN                            NaN   

                                                       col 

In [98]:
for table in clean_tables[siren]:
    print(table.shape)
    print(table.head())

AttributeError: 'str' object has no attribute 'shape'